<a href="https://colab.research.google.com/github/hjalmarfriden/StatisticsProjects/blob/main/AlgorithmCollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Collection of functions and classes to be used in projects**

In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

In [ ]:
url = 'https://raw.githubusercontent.com/hjalmarfriden/StatisticsProjects/main/datafolder/titanic_train.csv'
titanic = pd.read_csv(url)

In [ ]:
titanic.loc[:, ['Embarked', 'Pclass', 'Parch', 'Fare']].head()

,Embarked,Pclass,Parch,Fare
0,S,3,0,7.2500
1,C,1,0,71.2833
2,S,3,0,7.9250
3,S,1,0,53.1000
4,S,3,0,8.0500


**Own version of the histogram algorithm**

In [ ]:
Parch_data = titanic.loc[:,["Parch"]]
Parch_series = Parch_data["Parch"]
Parch_list = Parch_series.tolist()

Fare_data = titanic.loc[:,["Fare"]]
Fare_series = Fare_data["Fare"]
Fare_list = Fare_series.tolist()

def histogram_intervals(data, bins):
  c = Counter(data)
  key_values = list(c.keys())
  if bins > len(list(c.keys())):
    y_values = []
    for i in range(len(list(c.keys()))):
      y_values.append(c[key_values[i]])
    return [key_values, y_values]
  else:
    range_data = max(data) - min(data)
    bin_size = range_data/bins
    threshold = min(data) + bin_size
    interval_values = [min(data)]
    #Pre allocating since we want to refere to place
    y_values = [0]*bins
    for i in range(1, bins + 1):
      #This is which what rate the intervals grow
      threshold = interval_values[i - 1] + bin_size
      interval_values.append(threshold)
    
    for j in range(len(key_values)):
      for k in range(1, len(interval_values)):
        if key_values[j] >= interval_values[k - 1] and key_values[j] < interval_values[k]:
          tmp = c[key_values[j]] + y_values[k-1]
          y_values[k-1] = tmp
    #The last part in key_values will be as big as largest interval value
    #corner case, will add manually
    tmp = c[key_values[-1]] + y_values[-1]
    y_values[-1] = tmp
  return [interval_values, y_values]

fare_hist = histogram_intervals(Fare_list, 20)
print(fare_hist[1])
print(fare_hist[0])
print(np.histogram(Fare_list, bins=20))

print("NOW WE TEST FOR PARCH")

parch_hist = histogram_intervals(Parch_list, 3)
print(parch_hist[1])
print(parch_hist[0])
print(np.histogram(Parch_list, bins=3))

[562, 170, 67, 39, 15, 16, 2, 0, 9, 2, 6, 0, 0, 0, 0, 0, 0, 0, 0, 4]
[0.0, 25.61646, 51.23292, 76.84938, 102.46584, 128.0823, 153.69876, 179.31521999999998, 204.93167999999997, 230.54813999999996, 256.16459999999995, 281.78105999999997, 307.39752, 333.01398, 358.63044, 384.24690000000004, 409.86336000000006, 435.4798200000001, 461.0962800000001, 486.7127400000001, 512.3292000000001]
(array([562, 170,  67,  39,  15,  16,   2,   0,   9,   2,   6,   0,   0,
         0,   0,   0,   0,   0,   0,   3]), array([  0.     ,  25.61646,  51.23292,  76.84938, 102.46584, 128.0823 ,
       153.69876, 179.31522, 204.93168, 230.54814, 256.1646 , 281.78106,
       307.39752, 333.01398, 358.63044, 384.2469 , 409.86336, 435.47982,
       461.09628, 486.71274, 512.3292 ]))
NOW WE TEST FOR PARCH
[796, 85, 10]
[0, 2.0, 4.0, 6.0]
(array([796,  85,  10]), array([0., 2., 4., 6.]))


**Own version of the quantile algorithm**



In [ ]:
#Our own quantile method
def quantile(v,  p):
  n = len(v)
  sorted_v = sorted(v)
  if n % 2 == 1:
    #if uneven, return int rounded down
    index_pos = math.floor(p*n)
    return sorted_v[index_pos]
  else:
    #If even
    for i in range(len(v)):
      if i/n > p:
        #minus 1 since index start at 0
        res = sorted_v[i-1]
        res1 = res + p
        return res1
    else:
      #p must be 1
      return sorted_v[-1]

Fare_data = titanic.loc[:,["Fare"]]
Fare_series = Fare_data["Fare"]
Fare_list = Fare_series.tolist()
quantile(Fare_list, 0.5)
assert np.quantile(Fare_list, 0.5) == quantile(Fare_list, 0.5)
assert np.quantile(Fare_list, 0.2) == quantile(Fare_list, 0.2)
print("They give the same output")

They give the same output


**Own version of Naive Bayes classifier**
- Creating the algorithm
- Testing the algorithm
Expecting around 60% accuracy due to the weakness following the independece assumption.

In [ ]:
import math
import pandas as pd
import numpy as np
from scipy.stats import norm
from collections import Counter
import matplotlib.pyplot as plt
#For the import
from codecs import open
from __future__ import division


In [11]:
from google.colab import drive

In [13]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
plt.style.use('ggplot')

In [ ]:
def read_documents(doc_file):
    docs = []
    labels = []
    topic = []
    with open(doc_file, encoding='utf-8') as f:
        for line in f:
            words = line.strip().split()
            docs.append(words[3:])
            topic.append((words[0]))
            labels.append(words[1])
    return docs, labels, topic

In [14]:
all_docs, all_labels, all_topics = read_documents('/content/drive/MyDrive/Colab Notebooks/Github Public Project/all_sentiment_shuffled.txt')
split_point = int(0.80*len(all_docs))
train_docs = all_docs[:split_point]
train_labels = all_labels[:split_point]  
val_docs = all_docs[split_point:]
val_labels = all_labels[split_point:]

In [15]:
labels_dummy = []
for i in train_labels:
  if i == 'pos':
    labels_dummy.append(1)
  else:
    labels_dummy.append(0)

labels_dummy_val = []
for i in val_labels:
  if i == 'pos':
    labels_dummy_val.append(1)
  else:
    labels_dummy_val.append(0)

In [16]:
def train_nb(documents, labels):
  #Assuming we are taking in 0/1 for labels
  prior_pos = sum(labels)/len(labels)
  prior_neg = (len(labels) - prior_pos)/len(labels)
  pos_freq = Counter()
  neg_freq = Counter()
  for i in range(len(documents)):
    if labels[i] == 1:
      pos_freq.update(documents[i])
    else:
      neg_freq.update(documents[i])
  return pos_freq, prior_pos, neg_freq, prior_neg

In [17]:
parameters = train_nb(train_docs, labels_dummy)
def score_doc_label(document, parameters):
  pos_freq = parameters[0]
  prior_pos = parameters[1]
  neg_freq = parameters[2]
  prior_neg = parameters[3]
  pos_unique = sum(list(pos_freq.values()))
  neg_unique = sum(list(neg_freq.values()))
  score_pos = 0
  score_neg = 0
  for x in document:
    score_pos += np.log(1 + pos_freq[x]/pos_unique)
    score_neg += np.log(1 + neg_freq[x]/neg_unique)
  return np.exp(score_pos), np.exp(score_neg)

In [18]:
test = ['a', 'top-quality', 'performance']
score_doc_label(test, parameters)[0]

1.022730664434758

In [19]:
def classify_nb(document, parameters):
  return score_doc_label(document, parameters)[0] > score_doc_label(document, parameters)[1]

Evaluating the classifier

In [20]:
def classify_document(docs, parameters):
  guessed_labels = []
  for x in docs:
    if classify_nb(x, parameters):
      guessed_labels.append(1)
    else:
      guessed_labels.append(0)
  return guessed_labels

In [21]:
guessed_labels = classify_document(val_docs, parameters)

In [22]:
def f_accuracy(true_labels, guessed_labels):
  denom = len(guessed_labels)
  num = 0
  for i in range(len(true_labels)):
    if true_labels[i] == guessed_labels[i]:
      num += 1
  return num/denom

In [23]:
print('Accuracy is: ' + str(f_accuracy(labels_dummy_val, guessed_labels)))

Accuracy is: 0.6017624842635334


In [24]:
def f_precision(true_labels, guessed_labels):
  TP = 0
  for i in range(len(true_labels)):
    if true_labels[i] == guessed_labels[i] and true_labels[i] == 1:
      TP += 1
  if sum(true_labels) == 0 and TP != 0:
    return 0
  elif sum(true_labels) == 0 and TP == 0:
    return 1
  else:
    return TP/sum(true_labels)

In [25]:
def f_recall(true_labels, guessed_labels):
  TP = 0
  for i in range(len(true_labels)):
    if true_labels[i] == guessed_labels[i] and true_labels[i] == 1:
      TP += 1
  if sum(true_labels) == 0 and TP != 0:
    return 0
  elif sum(true_labels) == 0 and TP == 0:
    return 1
  else:
    return TP/sum(true_labels)

In [27]:
f_precision(labels_dummy_val, guessed_labels)

0.7649609713790113

In [28]:
def f_f1_score(true_labels, guessed_labels):
  precision = f_precision(true_labels, guessed_labels)
  recall = f_recall(true_labels, guessed_labels)
  if precision == 0 and recall == 0:
    return 0
  else:
    return 2*(precision*recall)/(precision+recall)

In [29]:
print("F1 score is " + str(round(f_f1_score(labels_dummy_val, guessed_labels),3)*100) + "%")
print("recall score is " + str(round(f_recall(labels_dummy_val, guessed_labels),3)*100) + "%")
print("precision score is " + str(round(f_precision(labels_dummy_val, guessed_labels),3)*100) + "%")

F1 score is 76.5%
recall score is 76.5%
precision score is 76.5%
